### LightFM Data Preparation

To work with the LightFM model example, we need to restructure the EBNeRD data.

How it is done in this notebook (for both training and validation data):
1. Load history and behaviour data
2. Make temporary dataframe containing all article_ids_inview for every unique user, because they are scattered in the behaviour dataframe
3. Append article_ids_inview to history dataframe now that they are grouped by user_id
4. Make new column for unclicked articles by comparing clicked and inview articles in history dataframe
5. Remove unnecessary columns
6. Restructure data, so every interaction (click and no click) has a single row, and generate ratings column based on click or no click (1 for click, 0 for no click)
7. Add topics column from article data using article_id (there are multiple topics for a single article)
8. Only keep the first topic for each article, and remove rows where topic is empty (and rename the column genre for now)
9. If there are any duplicate entries for any userID itemID combination, remove the duplicates
9. (Only for validation) Because the data is faulty, there are rows that are both present in training data and validation data. We remove these from the validation data.
10. Save data as csv

The data now looks like:
| userID | itemID | rating | genre  |
|--------|--------|--------|--------|
| 123456 | 518008 |   1    | Sport  |

At every critical step, sanity checks are in place to check for correctness.

### Shell

In [1]:
%pip install pandas
%pip install pyarrow
%pip install fastparquet

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pandas in d:\anaconda\envs\ml\lib\site-packages (2.2.1)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Imports

In [2]:
import pandas as pd

### Load article data

In [3]:
# Load EBNeRD news dataset
news = pd.read_parquet("./ebnerd_small/articles.parquet")
news.head()

,article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,...,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
0,3001353,Natascha var ikke den første,"Politiet frygter nu, at Nataschas bortfører ha...",2023-06-29 06:20:33,False,Sagen om den østriske Natascha og hendes bortf...,2006-08-31 08:06:45,[3150850],article_default,https://ekstrabladet.dk/krimi/article3001353.ece,...,[],"[Kriminalitet, Personfarlig kriminalitet]",140,[],krimi,NaN,NaN,NaN,0.9955,Negative
1,3003065,Kun Star Wars tjente mere,Biografgængerne strømmer ind for at se 'Da Vin...,2023-06-29 06:20:35,False,Vatikanet har opfordret til at boykotte filmen...,2006-05-21 16:57:00,[3006712],article_default,https://ekstrabladet.dk/underholdning/filmogtv...,...,[],"[Underholdning, Film og tv, Økonomi]",414,"[433, 434]",underholdning,NaN,NaN,NaN,0.8460,Positive
2,3012771,Morten Bruun fyret i SønderjyskE,FODBOLD: Morten Bruun fyret med øjeblikkelig v...,2023-06-29 06:20:39,False,Kemien mellem spillerne i Superligaklubben Søn...,2006-05-01 14:28:40,[3177953],article_default,https://ekstrabladet.dk/sport/fodbold/dansk_fo...,...,[],"[Erhverv, Kendt, Sport, Fodbold, Ansættelsesfo...",142,"[196, 199]",sport,NaN,NaN,NaN,0.8241,Negative
3,3023463,Luderne flytter på landet,I landets tyndest befolkede områder skyder bor...,2023-06-29 06:20:43,False,Det frække erhverv rykker på landet. I den tyn...,2007-03-24 08:27:59,[3184029],article_default,https://ekstrabladet.dk/nyheder/samfund/articl...,...,[],"[Livsstil, Erotik]",118,[133],nyheder,NaN,NaN,NaN,0.7053,Neutral
4,3032577,Cybersex: Hvornår er man utro?,En flirtende sms til den flotte fyr i regnskab...,2023-06-29 06:20:46,False,"De fleste af os mener, at et tungekys er utros...",2007-01-18 10:30:37,[3030463],article_default,https://ekstrabladet.dk/sex_og_samliv/article3...,...,[],"[Livsstil, Partnerskab]",565,[],sex_og_samliv,NaN,NaN,NaN,0.9307,Neutral


### Preparing train data with sanity checks

#### Load data

In [4]:
# Load EBNeRD history dataset for both train and validation
train_history = pd.read_parquet("./ebnerd_small/train/history.parquet")
train_history.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,13538,"[2023-04-27T10:17:43.000000, 2023-04-27T10:18:...","[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11..."
1,14241,"[2023-04-27T09:40:18.000000, 2023-04-27T09:40:...","[100.0, 46.0, 100.0, 70.0, 100.0, 100.0, 100.0...","[9738557, 9738528, 9738533, 9738684, 9739035, ...","[8.0, 9.0, 28.0, 17.0, 91.0, 21.0, 14.0, 27.0,..."
2,20396,"[2023-04-27T12:30:44.000000, 2023-04-27T12:31:...","[100.0, 59.0, nan, nan, 100.0, 100.0, nan, nan...","[9738760, 9738355, 9738355, 9739864, 9741788, ...","[49.0, 34.0, 0.0, 60.0, 180.0, 49.0, 0.0, 0.0,..."
3,34912,"[2023-04-29T07:12:49.000000, 2023-04-29T13:01:...","[100.0, 35.0, 44.0, 31.0, 100.0, 100.0, 100.0,...","[9741802, 9741804, 9741803, 9740087, 9742039, ...","[153.0, 7.0, 5.0, 6.0, 44.0, 44.0, 108.0, 10.0..."
4,37953,"[2023-04-27T19:17:10.000000, 2023-04-27T19:17:...","[14.0, 28.0, 29.0, nan, 36.0, 33.0, 50.0, 100....","[9739205, 9739202, 9737084, 9739274, 9739358, ...","[4.0, 16.0, 4.0, 0.0, 5.0, 5.0, 25.0, 48.0, 6...."


In [5]:
# Load EBNeRD behaviors dataset for both train and validation
train_behaviour = pd.read_parquet("./ebnerd_small/train/behaviors.parquet")
train_behaviour.head()

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,149474,NaN,2023-05-24 07:47:53,13.0,NaN,2,"[9778623, 9778682, 9778669, 9778657, 9778736, ...",[9778657],139836,False,NaN,NaN,NaN,False,759,7.0,22.0
1,150528,NaN,2023-05-24 07:33:25,25.0,NaN,2,"[9778718, 9778728, 9778745, 9778669, 9778657, ...",[9778623],143471,False,NaN,NaN,NaN,False,1240,287.0,100.0
2,153068,9778682.0,2023-05-24 07:09:04,78.0,100.0,1,"[9778657, 9778669, 9772866, 9776259, 9756397, ...",[9778669],151570,False,NaN,NaN,NaN,False,1976,45.0,100.0
3,153070,9777492.0,2023-05-24 07:13:14,26.0,100.0,1,"[9020783, 9778444, 9525589, 7213923, 9777397, ...",[9778628],151570,False,NaN,NaN,NaN,False,1976,4.0,18.0
4,153071,9778623.0,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, 9565836, 9335113, 9771223, ...",[9777492],151570,False,NaN,NaN,NaN,False,1976,26.0,100.0


#### Make temporary dataframe and group article_ids_inview for unique user_ids (they are scattered in behaviour dataframe)

In [6]:
# Group by user_id, flatten article_ids_inview lists, and remove duplicates
train_inview_temp = train_behaviour.groupby('user_id')['article_ids_inview'].agg(lambda x: list(set(y for sublist in x for y in sublist))).reset_index()

# Renaming columns
train_inview_temp.columns = ['user_id', 'article_ids_inview']

train_inview_temp.head()

,user_id,article_ids_inview
0,10068,"[9780096, 9772548, 9777156, 9758734, 9779860, ..."
1,10200,"[9772545, 9769474, 9268227, 9771009, 9774598, ..."
2,10201,"[9772032, 9772038, 9772168, 9772045, 9772047, ..."
3,10623,"[9774595, 9773574, 9769996, 9770515, 9744403, ..."
4,10701,"[9773070, 9773078, 9512983, 9773084, 9717791, ..."


##### Sanity checks

In [7]:
# Check for missing user IDs in history and behavior dataframes
# Print the number of unique user IDs in history and behaviors dataframes
print("Number of unique user IDs in history dataframe:", train_history['user_id'].nunique())
print("Number of unique user IDs in behaviors dataframe:", train_behaviour['user_id'].nunique())
print("Number of unique user IDs in inview_temp dataframe:", train_inview_temp['user_id'].nunique())

# Check for duplicate user IDs
duplicate_user_ids = train_history[train_history.duplicated('user_id')]['user_id'].unique()

if len(duplicate_user_ids) > 0:
    print("Duplicate user IDs found in history dataframe:")
    print(duplicate_user_ids)
else:
    print("No duplicate user IDs found in history dataframe.")

# Check if all user_ids match
# Get the set of unique user IDs in inview_temp dataframe
inview_temp_user_ids = set(train_inview_temp['user_id'])

# Get the set of unique user IDs in train_history dataframe
train_history_user_ids = set(train_history['user_id'])

# Check if the sets of unique user IDs are the same
user_ids_match = inview_temp_user_ids == train_history_user_ids

print("Distinct user IDs in inview_temp match with train_history:", user_ids_match)

# Inview_temp checks
# Check 1: Number of rows matches with number of rows in history dataframe
check1 = len(train_inview_temp) == len(train_history)

# Check 2: All user_ids are distinct
check2 = train_inview_temp['user_id'].nunique() == len(train_inview_temp)

# Check 3: Article_ids_inview contain no duplicates
check3 = all(len(set(article_ids)) == len(article_ids) for article_ids in train_inview_temp['article_ids_inview'])

# Printing the results of checks
print("Check 1: Number of rows match with number of rows in history dataframe:", check1)
print("Check 2: All user_ids are distinct:", check2)
print("Check 3: Article_ids_inview contain no duplicates:", check3)

Number of unique user IDs in history dataframe: 15143
Number of unique user IDs in behaviors dataframe: 15143
Number of unique user IDs in inview_temp dataframe: 15143
No duplicate user IDs found in history dataframe.
Distinct user IDs in inview_temp match with train_history: True
Check 1: Number of rows match with number of rows in history dataframe: True
Check 2: All user_ids are distinct: True
Check 3: Article_ids_inview contain no duplicates: True


#### We can now add article_ids_inview to history, because they are grouped in lists for unique user_ids

In [8]:
train_history_with_inview = pd.merge(train_history, train_inview_temp, on='user_id', how='left')
train_history_with_inview.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed,article_ids_inview
0,13538,"[2023-04-27T10:17:43.000000, 2023-04-27T10:18:...","[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...","[9773070, 9773078, 9775142, 9770028, 9754160, ..."
1,14241,"[2023-04-27T09:40:18.000000, 2023-04-27T09:40:...","[100.0, 46.0, 100.0, 70.0, 100.0, 100.0, 100.0...","[9738557, 9738528, 9738533, 9738684, 9739035, ...","[8.0, 9.0, 28.0, 17.0, 91.0, 21.0, 14.0, 27.0,...","[9772032, 9772038, 9772045, 9772047, 9772050, ..."
2,20396,"[2023-04-27T12:30:44.000000, 2023-04-27T12:31:...","[100.0, 59.0, nan, nan, 100.0, 100.0, nan, nan...","[9738760, 9738355, 9738355, 9739864, 9741788, ...","[49.0, 34.0, 0.0, 60.0, 180.0, 49.0, 0.0, 0.0,...","[9772545, 9769474, 9772548, 9779723, 9779737, ..."
3,34912,"[2023-04-29T07:12:49.000000, 2023-04-29T13:01:...","[100.0, 35.0, 44.0, 31.0, 100.0, 100.0, 100.0,...","[9741802, 9741804, 9741803, 9740087, 9742039, ...","[153.0, 7.0, 5.0, 6.0, 44.0, 44.0, 108.0, 10.0...","[9774595, 9772038, 9769996, 9772557, 9773070, ..."
4,37953,"[2023-04-27T19:17:10.000000, 2023-04-27T19:17:...","[14.0, 28.0, 29.0, nan, 36.0, 33.0, 50.0, 100....","[9739205, 9739202, 9737084, 9739274, 9739358, ...","[4.0, 16.0, 4.0, 0.0, 5.0, 5.0, 25.0, 48.0, 6....","[9775361, 9771779, 9769605, 9080070, 9775371, ..."


#### Make new column for unclicked articles

In [9]:
# Function to filter out non-clicked articles
def get_non_clicked_articles(inview_articles, clicked_articles):
    return [article for article in inview_articles if article not in clicked_articles]

In [10]:
# Apply the function to create the new column
train_history_with_inview['non_clicked_articles'] = train_history_with_inview.apply(lambda row: get_non_clicked_articles(row['article_ids_inview'], row['article_id_fixed']), axis=1)

# Display the updated dataframe
train_history_with_inview.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed,article_ids_inview,non_clicked_articles
0,13538,"[2023-04-27T10:17:43.000000, 2023-04-27T10:18:...","[100.0, 35.0, 100.0, 24.0, 100.0, 23.0, 100.0,...","[9738663, 9738569, 9738663, 9738490, 9738663, ...","[17.0, 12.0, 4.0, 5.0, 4.0, 9.0, 5.0, 46.0, 11...","[9773070, 9773078, 9775142, 9770028, 9754160, ...","[9773070, 9773078, 9775142, 9770028, 9754160, ..."
1,14241,"[2023-04-27T09:40:18.000000, 2023-04-27T09:40:...","[100.0, 46.0, 100.0, 70.0, 100.0, 100.0, 100.0...","[9738557, 9738528, 9738533, 9738684, 9739035, ...","[8.0, 9.0, 28.0, 17.0, 91.0, 21.0, 14.0, 27.0,...","[9772032, 9772038, 9772045, 9772047, 9772050, ...","[9772032, 9772038, 9772045, 9772047, 9772050, ..."
2,20396,"[2023-04-27T12:30:44.000000, 2023-04-27T12:31:...","[100.0, 59.0, nan, nan, 100.0, 100.0, nan, nan...","[9738760, 9738355, 9738355, 9739864, 9741788, ...","[49.0, 34.0, 0.0, 60.0, 180.0, 49.0, 0.0, 0.0,...","[9772545, 9769474, 9772548, 9779723, 9779737, ...","[9772545, 9769474, 9772548, 9779723, 9779737, ..."
3,34912,"[2023-04-29T07:12:49.000000, 2023-04-29T13:01:...","[100.0, 35.0, 44.0, 31.0, 100.0, 100.0, 100.0,...","[9741802, 9741804, 9741803, 9740087, 9742039, ...","[153.0, 7.0, 5.0, 6.0, 44.0, 44.0, 108.0, 10.0...","[9774595, 9772038, 9769996, 9772557, 9773070, ...","[9774595, 9772038, 9769996, 9772557, 9773070, ..."
4,37953,"[2023-04-27T19:17:10.000000, 2023-04-27T19:17:...","[14.0, 28.0, 29.0, nan, 36.0, 33.0, 50.0, 100....","[9739205, 9739202, 9737084, 9739274, 9739358, ...","[4.0, 16.0, 4.0, 0.0, 5.0, 5.0, 25.0, 48.0, 6....","[9775361, 9771779, 9769605, 9080070, 9775371, ...","[9775361, 9771779, 9769605, 9080070, 9775371, ..."


#### Remove unnecessary columns

In [11]:
# Selecting only the desired columns
train_history_with_unclicked = train_history_with_inview[['user_id', 'article_id_fixed', 'non_clicked_articles']]

# Display the updated dataframe
train_history_with_unclicked.head()

,user_id,article_id_fixed,non_clicked_articles
0,13538,"[9738663, 9738569, 9738663, 9738490, 9738663, ...","[9773070, 9773078, 9775142, 9770028, 9754160, ..."
1,14241,"[9738557, 9738528, 9738533, 9738684, 9739035, ...","[9772032, 9772038, 9772045, 9772047, 9772050, ..."
2,20396,"[9738760, 9738355, 9738355, 9739864, 9741788, ...","[9772545, 9769474, 9772548, 9779723, 9779737, ..."
3,34912,"[9741802, 9741804, 9741803, 9740087, 9742039, ...","[9774595, 9772038, 9769996, 9772557, 9773070, ..."
4,37953,"[9739205, 9739202, 9737084, 9739274, 9739358, ...","[9775361, 9771779, 9769605, 9080070, 9775371, ..."


#### Sanity check

In [12]:
# Assuming you have the train_history_with_unclicked dataframe

# Check if any element in article_id_fixed is present in non_clicked_articles for each row
rows_with_common_elements = train_history_with_unclicked[
    train_history_with_unclicked.apply(
        lambda row: any(item in row['article_id_fixed'] for item in row['non_clicked_articles']),
        axis=1
    )
]

# Check if there are any rows with common elements
if not rows_with_common_elements.empty:
    print("There are rows where article_id_fixed and non_clicked_articles contain the same element.")
    print(rows_with_common_elements)
else:
    print("No rows where article_id_fixed and non_clicked_articles contain the same element.")


No rows where article_id_fixed and non_clicked_articles contain the same element.


#### Generate ratings and restructure data

In [13]:
# Initialize an empty list to store the data
data = []

# Iterate over each row in merged_df_filtered
for index, row in train_history_with_unclicked.iterrows():
    # For each item in article_id_fixed, add a row with rating 5
    for item in row['article_id_fixed']:
        data.append([row['user_id'], item, 1])
    
    # For each item in non_clicked_articles, add a row with rating 1
    for item in row['non_clicked_articles']:
        data.append([row['user_id'], item, 0])

# Create the final dataframe from the collected data
train_history_with_rating = pd.DataFrame(data, columns=['userID', 'itemID', 'rating'])

# Display the first few rows of the final dataframe
train_history_with_rating.head()

,userID,itemID,rating
0,13538,9738663,1
1,13538,9738569,1
2,13538,9738663,1
3,13538,9738490,1
4,13538,9738663,1


##### Sanity check

In [14]:
# Calculate the total number of items in article_id_fixed and non_clicked_articles columns
total_items = train_history_with_unclicked['article_id_fixed'].apply(len).sum() + train_history_with_unclicked['non_clicked_articles'].apply(len).sum()

# Check if the total number of items is equal to the number of rows in final_data dataframe
check = len(train_history_with_rating) == total_items

# Print the result of the check
print("Check if the number of rows in data with rating equals\nthe total number of items in article_id_fixed and non_clicked_articles:", check)

Check if the number of rows in data with rating equals
the total number of items in article_id_fixed and non_clicked_articles: True


#### Add topics

In [15]:
# Merge final_data with news on article_id and item_id
train_history_with_topics = pd.merge(train_history_with_rating, news[['article_id', 'topics']], left_on='itemID', right_on='article_id', how='left')

# Drop the redundant article_id column
train_history_with_topics.drop(columns=['article_id'], inplace=True)

# Display the first few rows of the final_data_with_genre dataframe
train_history_with_topics.head()

,userID,itemID,rating,topics
0,13538,9738663,1,"[Erhverv, Privat virksomhed, Ansættelsesforhol..."
1,13538,9738569,1,"[Erhverv, Samfund, Sport, Bæredygtighed og klima]"
2,13538,9738663,1,"[Erhverv, Privat virksomhed, Ansættelsesforhol..."
3,13538,9738490,1,"[Erhverv, Privat virksomhed, Film og tv, Økono..."
4,13538,9738663,1,"[Erhverv, Privat virksomhed, Ansættelsesforhol..."


#### Remove rows without topics

In [16]:
# Remove rows where the 'topics' column contains empty arrays
train_history_with_topics = train_history_with_topics[train_history_with_topics['topics'].apply(len) > 0]

# Check if there are any rows where the 'topics' column contains empty arrays after removal
empty_topics_rows_after_removal = train_history_with_topics['topics'].apply(lambda x: len(x) == 0).any()

# Print the result
if empty_topics_rows_after_removal:
    print("There are still rows where the 'topics' column contains empty arrays after removal.")
else:
    print("All rows with empty arrays in the 'topics' column have been successfully removed.")


All rows with empty arrays in the 'topics' column have been successfully removed.


#### Sanity check

In [17]:
# Filter rows where the 'topics' column contains empty arrays
rows_with_empty_topics = train_history_with_topics[train_history_with_topics['topics'].apply(len) == 0]

# Print the rows
if not rows_with_empty_topics.empty:
    print("Rows where the 'topics' column contains empty arrays:")
    print(rows_with_empty_topics)
else:
    print("No rows where the 'topics' column contains empty arrays.")


No rows where the 'topics' column contains empty arrays.


#### Add genre (first element from topics) and remove topics

In [18]:
def extract_genre(topics):
    if len(topics) > 0:
        return topics[0]
    else:
        return None

# Create the new "genre" column
train_history_with_topics['genre'] = train_history_with_topics['topics'].apply(extract_genre)

# Drop the redundant "topics" column
train_history_with_topics.drop(columns=['topics'], inplace=True)

# Display the first few rows of the updated dataframe
train_history_with_topics.head()

,userID,itemID,rating,genre
0,13538,9738663,1,Erhverv
1,13538,9738569,1,Erhverv
2,13538,9738663,1,Erhverv
3,13538,9738490,1,Erhverv
4,13538,9738663,1,Erhverv


#### Remove any duplicate rows

In [19]:
# Check for duplicate rows
duplicate_rows = train_history_with_topics.duplicated().sum()

# Output the number of duplicate rows
print("Number of duplicate rows:", duplicate_rows)

# Remove duplicate rows from the dataframe
train_history_with_topics = train_history_with_topics.drop_duplicates()

# Reset the indices
train_history_with_topics.reset_index(drop=True, inplace=True)

# Output the updated dataframe
print("Updated dataframe:")
print(train_history_with_topics)

Number of duplicate rows: 372562
Updated dataframe:
          userID   itemID  rating         genre
0          13538  9738663       1       Erhverv
1          13538  9738569       1       Erhverv
2          13538  9738490       1       Erhverv
3          13538  9738667       1       Erhverv
4          13538  9738528       1      Livsstil
...          ...      ...     ...           ...
3845084  1710834  9771764       0  Kriminalitet
3845085  1710834  9768308       0         Kendt
3845086  1710834  9695098       0        Kultur
3845087  1710834  9769341       0         Kendt
3845088  1710834  9771775       0    Begivenhed

[3845089 rows x 4 columns]


#### Sanity check

In [20]:
# Group the dataframe by userID and itemID and count occurrences
duplicate_rows = train_history_with_topics.groupby(['userID', 'itemID']).size().reset_index(name='count')

# Filter out rows where count is greater than 1
duplicate_rows = duplicate_rows[duplicate_rows['count'] > 1]

# Output the duplicate rows
print("Duplicate rows with matching userID and itemID:")
print(duplicate_rows)

Duplicate rows with matching userID and itemID:
Empty DataFrame
Columns: [userID, itemID, count]
Index: []


### Save data

In [21]:
# Specify the file path where you want to save the CSV file
file_path = "exported_data/train_data.csv"

# Save the final_data dataframe to a CSV file
train_history_with_topics.to_csv(file_path, index=False)

print("Data saved to", file_path)

Data saved to exported_data/train_data.csv


### Repeat same steps for validation data

In [22]:
valid_history = pd.read_parquet("./ebnerd_small/validation/history.parquet")
valid_history.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,14241,"[2023-05-04T12:42:38.000000, 2023-05-04T12:43:...","[100.0, 77.0, 100.0, 100.0, 59.0, 61.0, 100.0,...","[9578072, 9749349, 9750161, 9749076, 9749947, ...","[20.0, 16.0, 17.0, 31.0, 4.0, 14.0, 13.0, 19.0..."
1,20396,"[2023-05-04T10:09:26.000000, 2023-05-04T10:47:...","[nan, 100.0, 100.0, nan, 100.0, nan, 15.0, 100...","[9748482, 9748916, 9749156, 9749278, 9745750, ...","[0.0, 69.0, 33.0, 0.0, 95.0, 11.0, 4.0, 88.0, ..."
2,37953,"[2023-05-04T20:51:41.000000, 2023-05-04T20:51:...","[25.0, 13.0, 66.0, 71.0, 54.0, nan, 28.0, 94.0...","[9750397, 9749916, 9750358, 9751284, 9749284, ...","[5.0, 5.0, 7.0, 182.0, 7.0, 0.0, 3.0, 14.0, 34..."
3,38910,"[2023-05-04T13:41:54.000000, 2023-05-04T13:41:...","[nan, 23.0, nan, 18.0, 15.0, 100.0, 29.0, 26.0...","[9745491, 9749729, 9749495, 9749495, 9741788, ...","[2.0, 2.0, 86.0, 2.0, 3.0, 63.0, 2.0, 10.0, 59..."
4,39221,"[2023-05-06T03:54:52.000000, 2023-05-06T03:56:...","[100.0, 100.0, 64.0, 77.0, 100.0, nan, 100.0, ...","[9749034, 9751385, 9559425, 9751115, 9685759, ...","[67.0, 82.0, 98.0, 24.0, 26.0, 0.0, 62.0, 34.0..."


In [23]:
valid_behaviour = pd.read_parquet("./ebnerd_small/validation/behaviors.parquet")
valid_behaviour.head()

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,96791,NaN,2023-05-28 04:21:24,9.0,NaN,2,"[9783865, 9784591, 9784679, 9784696, 9784710]",[9784696],22548,False,NaN,NaN,NaN,False,142,72.0,100.0
1,96798,NaN,2023-05-28 04:31:48,46.0,NaN,2,"[9782884, 9783865, 9782726, 9695098, 9782806, ...",[9784281],22548,False,NaN,NaN,NaN,False,143,16.0,28.0
2,96801,NaN,2023-05-28 04:30:17,14.0,NaN,2,"[9784648, 7184889, 9784444, 9781785, 9783405, ...",[9784444],22548,False,NaN,NaN,NaN,False,143,12.0,24.0
3,96808,NaN,2023-05-28 04:27:19,22.0,NaN,2,"[9784607, 9695098, 9784406, 7184889, 9781983]",[9781983],22548,False,NaN,NaN,NaN,False,142,125.0,80.0
4,96810,NaN,2023-05-28 04:29:47,23.0,NaN,2,"[9781983, 7184889, 9784642, 9782108, 9781785, ...",[9784642],22548,False,NaN,NaN,NaN,False,142,NaN,NaN


In [24]:
valid_inview_temp = valid_behaviour.groupby('user_id')['article_ids_inview'].agg(lambda x: list(set(y for sublist in x for y in sublist))).reset_index()
valid_inview_temp.columns = ['user_id', 'article_ids_inview']
valid_inview_temp.head()

,user_id,article_ids_inview
0,10200,"[9783296, 9780736, 9779713, 9785349, 9785868, ..."
1,10201,"[9780928, 9781057, 9780993, 9780769, 9780267, ..."
2,10385,"[9533957, 9785868, 9783824, 9784856, 9781785, ..."
3,10623,"[9754112, 9784839, 9785829, 9790475, 9790476, ..."
4,10701,"[9785349, 9789446, 9781257, 9785361, 9783317, ..."


In [25]:
valid_history_with_inview = pd.merge(valid_history, valid_inview_temp, on='user_id', how='left')
valid_history_with_inview.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed,article_ids_inview
0,14241,"[2023-05-04T12:42:38.000000, 2023-05-04T12:43:...","[100.0, 77.0, 100.0, 100.0, 59.0, 61.0, 100.0,...","[9578072, 9749349, 9750161, 9749076, 9749947, ...","[20.0, 16.0, 17.0, 31.0, 4.0, 14.0, 13.0, 19.0...","[9486080, 9783042, 9458953, 9790475, 9790476, ..."
1,20396,"[2023-05-04T10:09:26.000000, 2023-05-04T10:47:...","[nan, 100.0, 100.0, nan, 100.0, nan, 15.0, 100...","[9748482, 9748916, 9749156, 9749278, 9745750, ...","[0.0, 69.0, 33.0, 0.0, 95.0, 11.0, 4.0, 88.0, ...","[9781763, 9784839, 9785868, 9339920, 9789974, ..."
2,37953,"[2023-05-04T20:51:41.000000, 2023-05-04T20:51:...","[25.0, 13.0, 66.0, 71.0, 54.0, nan, 28.0, 94.0...","[9750397, 9749916, 9750358, 9751284, 9749284, ...","[5.0, 5.0, 7.0, 182.0, 7.0, 0.0, 3.0, 14.0, 34...","[9782656, 8007684, 9784710, 9783183, 9784591, ..."
3,38910,"[2023-05-04T13:41:54.000000, 2023-05-04T13:41:...","[nan, 23.0, nan, 18.0, 15.0, 100.0, 29.0, 26.0...","[9745491, 9749729, 9749495, 9749495, 9741788, ...","[2.0, 2.0, 86.0, 2.0, 3.0, 63.0, 2.0, 10.0, 59...","[9783183, 9782800, 9364115, 9788183, 9779225, ..."
4,39221,"[2023-05-06T03:54:52.000000, 2023-05-06T03:56:...","[100.0, 100.0, 64.0, 77.0, 100.0, nan, 100.0, ...","[9749034, 9751385, 9559425, 9751115, 9685759, ...","[67.0, 82.0, 98.0, 24.0, 26.0, 0.0, 62.0, 34.0...","[9533957, 9785350, 9786378, 9790987, 9744396, ..."


In [26]:
valid_history_with_inview['non_clicked_articles'] = valid_history_with_inview.apply(lambda row: get_non_clicked_articles(row['article_ids_inview'], row['article_id_fixed']), axis=1)
valid_history_with_inview.head()

,user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed,article_ids_inview,non_clicked_articles
0,14241,"[2023-05-04T12:42:38.000000, 2023-05-04T12:43:...","[100.0, 77.0, 100.0, 100.0, 59.0, 61.0, 100.0,...","[9578072, 9749349, 9750161, 9749076, 9749947, ...","[20.0, 16.0, 17.0, 31.0, 4.0, 14.0, 13.0, 19.0...","[9486080, 9783042, 9458953, 9790475, 9790476, ...","[9486080, 9783042, 9458953, 9790475, 9790476, ..."
1,20396,"[2023-05-04T10:09:26.000000, 2023-05-04T10:47:...","[nan, 100.0, 100.0, nan, 100.0, nan, 15.0, 100...","[9748482, 9748916, 9749156, 9749278, 9745750, ...","[0.0, 69.0, 33.0, 0.0, 95.0, 11.0, 4.0, 88.0, ...","[9781763, 9784839, 9785868, 9339920, 9789974, ...","[9781763, 9784839, 9785868, 9339920, 9789974, ..."
2,37953,"[2023-05-04T20:51:41.000000, 2023-05-04T20:51:...","[25.0, 13.0, 66.0, 71.0, 54.0, nan, 28.0, 94.0...","[9750397, 9749916, 9750358, 9751284, 9749284, ...","[5.0, 5.0, 7.0, 182.0, 7.0, 0.0, 3.0, 14.0, 34...","[9782656, 8007684, 9784710, 9783183, 9784591, ...","[9782656, 8007684, 9784710, 9783183, 9784591, ..."
3,38910,"[2023-05-04T13:41:54.000000, 2023-05-04T13:41:...","[nan, 23.0, nan, 18.0, 15.0, 100.0, 29.0, 26.0...","[9745491, 9749729, 9749495, 9749495, 9741788, ...","[2.0, 2.0, 86.0, 2.0, 3.0, 63.0, 2.0, 10.0, 59...","[9783183, 9782800, 9364115, 9788183, 9779225, ...","[9783183, 9782800, 9364115, 9788183, 9779225, ..."
4,39221,"[2023-05-06T03:54:52.000000, 2023-05-06T03:56:...","[100.0, 100.0, 64.0, 77.0, 100.0, nan, 100.0, ...","[9749034, 9751385, 9559425, 9751115, 9685759, ...","[67.0, 82.0, 98.0, 24.0, 26.0, 0.0, 62.0, 34.0...","[9533957, 9785350, 9786378, 9790987, 9744396, ...","[9533957, 9785350, 9786378, 9790987, 9744396, ..."


In [27]:
# Selecting only the desired columns
valid_history_with_unclicked = valid_history_with_inview[['user_id', 'article_id_fixed', 'non_clicked_articles']]

# Display the updated dataframe
valid_history_with_unclicked.head()

,user_id,article_id_fixed,non_clicked_articles
0,14241,"[9578072, 9749349, 9750161, 9749076, 9749947, ...","[9486080, 9783042, 9458953, 9790475, 9790476, ..."
1,20396,"[9748482, 9748916, 9749156, 9749278, 9745750, ...","[9781763, 9784839, 9785868, 9339920, 9789974, ..."
2,37953,"[9750397, 9749916, 9750358, 9751284, 9749284, ...","[9782656, 8007684, 9784710, 9783183, 9784591, ..."
3,38910,"[9745491, 9749729, 9749495, 9749495, 9741788, ...","[9783183, 9782800, 9364115, 9788183, 9779225, ..."
4,39221,"[9749034, 9751385, 9559425, 9751115, 9685759, ...","[9533957, 9785350, 9786378, 9790987, 9744396, ..."


In [28]:
data = []

for index, row in valid_history_with_unclicked.iterrows():
    for item in row['article_id_fixed']:
        data.append([row['user_id'], item, 1])
    
    for item in row['non_clicked_articles']:
        data.append([row['user_id'], item, 0])

valid_history_with_rating = pd.DataFrame(data, columns=['userID', 'itemID', 'rating'])
valid_history_with_rating.head()

,userID,itemID,rating
0,14241,9578072,1
1,14241,9749349,1
2,14241,9750161,1
3,14241,9749076,1
4,14241,9749947,1


In [29]:
valid_history_with_topics = pd.merge(valid_history_with_rating, news[['article_id', 'topics']], left_on='itemID', right_on='article_id', how='left')
valid_history_with_topics.drop(columns=['article_id'], inplace=True)
valid_history_with_topics.head()

,userID,itemID,rating,topics
0,14241,9578072,1,"[Økonomi, Mikro, Makro, Bolig]"
1,14241,9749349,1,"[Kendt, Underholdning, Film og tv]"
2,14241,9750161,1,"[Kriminalitet, Kendt]"
3,14241,9749076,1,"[Erhverv, Privat virksomhed, Økonomi]"
4,14241,9749947,1,"[Erhverv, Økonomi, Makro]"


In [30]:
# Remove rows where the 'topics' column contains empty arrays
valid_history_with_topics = valid_history_with_topics[valid_history_with_topics['topics'].apply(len) > 0]

# Check if there are any rows where the 'topics' column contains empty arrays after removal
empty_topics_rows_after_removal = valid_history_with_topics['topics'].apply(lambda x: len(x) == 0).any()

# Print the result
if empty_topics_rows_after_removal:
    print("There are still rows where the 'topics' column contains empty arrays after removal.")
else:
    print("All rows with empty arrays in the 'topics' column have been successfully removed.")

All rows with empty arrays in the 'topics' column have been successfully removed.


In [31]:
valid_history_with_topics['genre'] = valid_history_with_topics['topics'].apply(extract_genre)
valid_history_with_topics.drop(columns=['topics'], inplace=True)
valid_history_with_topics.head()

,userID,itemID,rating,genre
0,14241,9578072,1,Økonomi
1,14241,9749349,1,Kendt
2,14241,9750161,1,Kriminalitet
3,14241,9749076,1,Erhverv
4,14241,9749947,1,Erhverv


In [32]:
# Check for duplicate rows
duplicate_rows = valid_history_with_topics.duplicated().sum()

# Output the number of duplicate rows
print("Number of duplicate rows:", duplicate_rows)

# Remove duplicate rows from the dataframe
valid_history_with_topics = valid_history_with_topics.drop_duplicates()

# Reset the indices
valid_history_with_topics.reset_index(drop=True, inplace=True)

# Output the updated dataframe
print("Updated dataframe:")
print(valid_history_with_topics)

Number of duplicate rows: 336758
Updated dataframe:
         userID   itemID  rating         genre
0         14241  9578072       1       Økonomi
1         14241  9749349       1         Kendt
2         14241  9750161       1  Kriminalitet
3         14241  9749076       1       Erhverv
4         14241  9749947       1       Erhverv
...         ...      ...     ...           ...
3834522  981701  9781624       0       Erhverv
3834523  981701  9779577       0       Erhverv
3834524  981701  9779066       0         Kendt
3834525  981701  9780476       0       Erhverv
3834526  981701  9781502       0  Kriminalitet

[3834527 rows x 4 columns]


#### Removing common rows between training and validation from the validation dataset

In [33]:
# Assuming you have imported pandas as pd
common_rows = pd.merge(valid_history_with_topics, train_history_with_topics, on=['userID', 'itemID'], how='inner')

if not common_rows.empty:
    print("There are common rows between the two dataframes.")
    print(common_rows)
else:
    print("There are no common rows between the two dataframes.")

common_indices = common_rows.index
# Remove common rows based on userID and itemID combination
valid_history_without_matches = valid_history_with_topics[
    ~valid_history_with_topics.set_index(['userID', 'itemID']).index.isin(common_rows.set_index(['userID', 'itemID']).index)
]

# Reset the index of valid_history_without_matches
valid_history_without_matches.reset_index(drop=True, inplace=True)

print(valid_history_without_matches)



There are common rows between the two dataframes.
          userID   itemID  rating_x       genre_x  rating_y       genre_y
0          14241  9578072         1       Økonomi         1       Økonomi
1          14241  9749349         1         Kendt         1         Kendt
2          14241  9750161         1  Kriminalitet         1  Kriminalitet
3          14241  9749076         1       Erhverv         1       Erhverv
4          14241  9749947         1       Erhverv         1       Erhverv
...          ...      ...       ...           ...       ...           ...
1532745  1909959  9776040         1  Kriminalitet         0  Kriminalitet
1532746  1909959  9695098         0        Kultur         0        Kultur
1532747  1106504  9778236         1    Katastrofe         0    Katastrofe
1532748  1106504  9778220         1       Erhverv         0       Erhverv
1532749  1106504  9777955         1      Livsstil         0      Livsstil

[1532750 rows x 6 columns]
         userID   itemID  rating  

#### Sanity check

In [34]:
common_rows_after_drop = pd.merge(valid_history_without_matches, train_history_with_topics, on=['userID', 'itemID'], how='inner')

if not common_rows_after_drop.empty:
    print("There are still common rows between the two dataframes after dropping.")
    print(common_rows_after_drop)
else:
    print("There are no common rows between the two dataframes after dropping.")

There are no common rows between the two dataframes after dropping.


In [35]:
file_path = "exported_data/valid_data.csv"
valid_history_with_topics.to_csv(file_path, index=False)
print("Data saved to", file_path)

Data saved to exported_data/valid_data.csv


### Combine train and valid data (not used but keep as utility)

#### Combine first 50.000 train data entries and 50.000 validation data entries for smaller dataset

In [36]:
SPLITNUM = 50000

# Select the first 50,000 rows of train_history_with_topics
train_subset = train_history_with_topics.iloc[:SPLITNUM]

# Select the first 50,000 rows of valid_history_with_topics
valid_subset = valid_history_with_topics.iloc[:SPLITNUM]

# Concatenate the subsets
combined_df = pd.concat([train_subset, valid_subset], ignore_index=True)

# Output the index where valid_history_with_topics starts
valid_start_index = len(train_subset)
print("Index where valid_history_with_topics starts:", valid_start_index)

Index where valid_history_with_topics starts: 50000


#### Sanity check

In [37]:
# Output the row at valid_start_index in combined_df
row_valid_start = combined_df.iloc[valid_start_index]
print("Row at valid_start_index in combined_df:")
print(row_valid_start)

# Output the first row of train_history_with_topics
first_row_train = valid_history_with_topics.iloc[0]
print("\nFirst row of valid_history_with_topics:")
print(first_row_train)

Row at valid_start_index in combined_df:
userID      14241
itemID    9578072
rating          1
genre     Økonomi
Name: 50000, dtype: object

First row of valid_history_with_topics:
userID      14241
itemID    9578072
rating          1
genre     Økonomi
Name: 0, dtype: object


#### Save combined data

In [38]:
file_path = "exported_data/combined_data.csv"
valid_history_with_topics.to_csv(file_path, index=False)
print("Data saved to", file_path)

Data saved to exported_data/combined_data.csv
